### Projectile

r, v, a, mass, friction.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [29]:
class Projectile:

	def __init__(self, r0, v0, m, t_span, a=np.array([0, -9.81]), c=np.array([0, 0])):

		self.r = np.array([r0])
		self.v = np.array([v0])
		self.m = m
		self.t_span = t_span
		self.a = a
		self.c = c

	def trajectory(self):
		dt = self.t_span[1] - self.t_span[0]
		for i in range(len(self.t_span)-1):
			v = self.v[-1] + self.a*dt + self.c*dt
			r = self.r[-1] + v*dt
			self.v = np.append(self.v, v, axis=0)
			self.r = np.append(self.r, r, axis=0)
		
		self.r = self.r.reshape(-1, 2)
		self.v = self.v.reshape(-1, 2)

	def peak(self):
		peak = np.argmax(self.r[:,1])
		return self.r[peak]
	
	def plot_trajectory(self):
		peak = self.peak()
		plt.plot(self.r[:,0], self.r[:,1], label='Trajectory')
		plt.plot(peak[0], peak[1], 'ro', label='Peak')
		plt.show()

In [22]:
p = Projectile(np.array([0, 0]), np.array([10, 10]), 1, np.linspace(0, 1, 100))
p.trajectory()

In [33]:
v = np.array([1, 2, 3, 4, 5, 6])
v.reshape(-1, 2)

array([[1, 2],
       [3, 4],
       [5, 6]])